In [2]:
import pandas as pd
from pathlib import Path

In [4]:
ROOT = Path.cwd().resolve().parent
DATA = ROOT / "Data" / "December1999.txt"

In [7]:
df = pd.read_csv(DATA, sep='\t')
print(df.head())

  vru+line  call_id  customer_id  priority type    date vru_entry vru_exit  \
0   AA0101    49296          0.0         0   PS  991201   5:20:04  5:20:41   
1   AA0101    49297   67472894.0         2   PS  991201   7:22:53  7:22:58   
2   AA0101    49298   56165871.0         2   PS  991201   7:44:58  7:45:04   
3   AA0101    49299          0.0         0   PS  991201   8:06:40  8:06:48   
4   AA0101    49300   42791442.0         2   PS  991201   8:24:13  8:24:19   

   vru_time  q_start   q_exit  q_time outcome ser_start ser_exit  ser_time  \
0        37  0:00:00  0:00:00       0    HANG   0:00:00  0:00:00         0   
1         5  7:22:58  7:24:18      80   AGENT   7:24:17  7:30:09       352   
2         6  7:45:04  7:48:22     198   AGENT   7:48:21  7:49:16        55   
3         8  0:00:00  0:00:00       0   AGENT   8:06:47  8:09:25       158   
4         6  8:24:19  8:25:00      41   AGENT   8:25:00  8:28:01       181   

      server  
0  NO_SERVER  
1     ZOHARI  
2     GELBER  
3 

Como queremos simular una M/M/S necesitamos datos en donde el server no sea NO_SERVER por lo que eliminaremos las filas con server: NO_SERVER

In [12]:
df = df[df['server'] != 'NO_SERVER']
df

,vru+line,call_id,customer_id,priority,type,date,vru_entry,vru_exit,vru_time,q_start,q_exit,q_time,outcome,ser_start,ser_exit,ser_time,server
1,AA0101,49297,67472894.0,2,PS,991201,7:22:53,7:22:58,5,7:22:58,7:24:18,80,AGENT,7:24:17,7:30:09,352,ZOHARI
2,AA0101,49298,56165871.0,2,PS,991201,7:44:58,7:45:04,6,7:45:04,7:48:22,198,AGENT,7:48:21,7:49:16,55,GELBER
3,AA0101,49299,0.0,0,PS,991201,8:06:40,8:06:48,8,0:00:00,0:00:00,0,AGENT,8:06:47,8:09:25,158,ROTH
4,AA0101,49300,42791442.0,2,PS,991201,8:24:13,8:24:19,6,8:24:19,8:25:00,41,AGENT,8:25:00,8:28:01,181,SHARON
5,AA0101,49301,7750219.0,2,PS,991201,8:33:24,8:33:30,6,8:33:30,8:33:55,25,AGENT,8:33:54,8:38:47,293,SHARON
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
43055,AA0216,17206,0.0,0,IN,991231,11:40:37,11:40:46,9,11:40:46,11:41:44,58,AGENT,11:41:43,11:51:52,609,Z2ARIE
43056,AA0216,17207,0.0,0,IN,991231,12:37:47,12:38:04,17,0:00:00,0:00:00,0,AGENT,12:38:03,12:41:07,184,Z2ARIE
43057,AA0216,17208,0.0,0,IN,991231,12:59:03,12:59:19,16,0:00:00,0:00:00,0,AGENT,12:59:18,13:05:47,389,Z2ARIE
43058,AA0216,17209,0.0,0,IN,991231,13:18:00,13:18:14,14,0:00:00,0:00:00,0,AGENT,13:18:13,13:18:30,17,Z2ARIE
